In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
from tqdm.notebook import tqdm

# Batching

In [3]:
class Classifier_1(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Classifier_1, self).__init__()
        self.i2h = nn.Linear(input_size, hidden_size) # W1x + b1
        self.h2o = nn.Linear(hidden_size, output_size) # w2h + b2
        self.relu = nn.ReLU()
    
    def forward(self, inp):
        x = self.i2h(inp) # inp @ self.W_1 can be replaced by torch.matmul(inp, self.W_1)
        h = self.relu(x)
        y = self.h2o(h)
        return y

In [4]:
criterion = nn.CrossEntropyLoss()

In [5]:
clf = Classifier_1(4, 10, 2)

In [ ]:
# We have till now only been backpropagating with just one example...
# In practice we optimize over a batch of input datapoints
# We will now see how to deal with batches
# Actually not much modification is needed

In [6]:
inp = torch.randn(6,4) # we have 6 data points each with 4 features

In [7]:
label = torch.LongTensor([1,0,0,1,1,0]) # labels for these random outputs

In [8]:
out = clf(inp)

In [9]:
out.shape # output corresponding to the 6 data points

torch.Size([6, 2])

In [8]:
out

tensor([[-0.2533, -0.0357],
        [ 0.1266, -0.1031],
        [-0.0775,  0.0793],
        [ 0.0043, -0.0679],
        [-0.5174,  0.2627],
        [-0.5508,  0.3134]], grad_fn=<AddmmBackward0>)

In [11]:
loss = criterion(out, label) # calculated across the whole batch (6 data points) and then calculating the mean

In [12]:
loss

tensor(0.7121, grad_fn=<NllLossBackward0>)

In [13]:
# the training is again similar
optimizer = optim.SGD(clf.parameters(),lr=0.001)
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [ ]:
# It is pretty easy to extend from a single data point to a batch...
# I often find it useful when designing a architecture to first design it for a single data point and then 
# extending it to batch

# Initialization and normalization

In [23]:
class Classifier_1(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Classifier_1, self).__init__()
        self.i2h = nn.Linear(input_size, hidden_size) # W1x + b1
        self.norm = nn.LayerNorm(hidden_size) # layer norm
        self.h2o = nn.Linear(hidden_size, output_size) # w2h + b2
        self.relu = nn.ReLU()
        # initilization
        nn.init.xavier_normal_(self.i2h.weight)
        nn.init.xavier_normal_(self.h2o.weight)
    
    def forward(self, inp):
        x = self.i2h(inp) # inp @ self.W_1 can be replaced by torch.matmul(inp, self.W_1)
        x = self.norm(x)
        h = self.relu(x)
        y = self.h2o(h)
        return y

In [24]:
clf = Classifier_1(4,6,2)

In [22]:
clf.i2h.bias

Parameter containing:
tensor([ 0.4971,  0.2784,  0.3415,  0.4611,  0.0142, -0.4708],
       requires_grad=True)

<img src="Images/mnist.png">

# Data preprocessing

In [ ]:
# We would like to train the model in batches...
# Ideally examples in a batch should be picked at random...
# Of course we can code it ourselves, but pytorch provides a Datset module to do just that...

https://drive.google.com/drive/folders/1y6_ddgZuxdMgHlM4BjZp9mmtwFWI7p4t?usp=sharing (Link to the dataset)

In [25]:
data_points = []
class_labels = []

with open('mnist_train.csv') as fs:
    for line in fs:
        data = list(map(int, line.strip().split(','))) 
        label = data[0]
        datapoint = data[1:]
        data_points.append(datapoint)
        class_labels.append(label)

In [26]:
len(class_labels)

60000

In [27]:
len(data_points[0])

784

In [28]:
from torch.utils.data import Dataset, DataLoader

In [29]:
class Mnist_dataset(Dataset):
    def __init__(self, data_points, class_labels):
        super(Dataset, self).__init__()
        self.data = data_points
        self.labels = class_labels
    
    def __len__(self):
        # returns length of the dataset
        return len(self.labels)
    
    def __getitem__(self, index):
        # retrieves an item of a given index
        d = torch.FloatTensor(self.data[index])
        l = torch.LongTensor([self.labels[index]])
        return d,l

In [30]:
mnist_data = Mnist_dataset(data_points, class_labels)
dataloader = DataLoader(mnist_data, batch_size=32, shuffle=True)
# this will resturn a batch of 32 examples which you can directly set as input to the model clf()

In [31]:
clf = Classifier_1(784, 1056, 10)

In [37]:
# initialize the optimizer
epochs = 10
optimizer = optim.SGD(clf.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
for e in tqdm(range(epochs)):
    for data, label in dataloader:
        out = clf(data)
        loss = criterion(out, label.squeeze(1))
        optimizer.zero_grad
        loss.backward()
        optimizer.step()

  0%|          | 0/10 [00:00<?, ?it/s]

In [38]:
test_data_points = []
test_class_labels = []

with open('mnist_test.csv') as fs:
    for line in fs:
        data = list(map(int, line.strip().split(','))) 
        label = data[0]
        datapoint = data[1:]
        test_data_points.append(datapoint)
        test_class_labels.append(label)

In [39]:
test_mnist_data = Mnist_dataset(test_data_points, test_class_labels)
testloader = DataLoader(test_mnist_data, batch_size=32)

In [41]:
ground_truth = []
predictions = []
with torch.no_grad():
    for data, label in testloader:
        out = clf(data)
        pred = torch.argmax(out, dim=1)
        ground_truth.extend(label.reshape(-1).numpy().tolist())
        predictions.extend(pred.reshape(-1).numpy().tolist())

In [42]:
from sklearn.metrics import accuracy_score

In [43]:
accuracy_score(ground_truth, predictions)

0.8098

# Saving and loading models

In [44]:
clf.state_dict()

OrderedDict([('i2h.weight',
              tensor([[ 0.0257,  0.0392, -0.0265,  ...,  0.0122,  0.0208,  0.0304],
                      [-0.0097,  0.0094, -0.0509,  ...,  0.0029,  0.0010,  0.0422],
                      [ 0.0114,  0.0229, -0.0210,  ...,  0.0053,  0.0013,  0.0514],
                      ...,
                      [-0.0017, -0.0256, -0.0152,  ...,  0.0431,  0.0117, -0.0037],
                      [ 0.0320,  0.0567, -0.0085,  ..., -0.0027,  0.0028, -0.0359],
                      [ 0.0106, -0.0102, -0.0372,  ...,  0.0429,  0.0526,  0.0190]])),
             ('i2h.bias',
              tensor([ 0.0346, -0.1071,  0.0148,  ..., -0.0476, -0.0288, -0.0131])),
             ('norm.weight',
              tensor([  1.1315, 123.8293,  -9.2225,  ...,   0.3094,  -6.8292,  18.3751])),
             ('norm.bias',
              tensor([-14.1297, -26.4977, -21.6721,  ...,  -0.9982, -16.9196,  -5.4343])),
             ('h2o.weight',
              tensor([[ 7.9275e+00,  1.0292e+01,  1.4945e+00,

In [47]:
torch.save(clf.state_dict(), 'model_mnist.pt')

In [48]:
model = Classifier_1(784, 1056, 10)

In [50]:
model = model.load_state_dict(torch.load('model_mnist.pt'))

# Tasks

We will consider the task of Human activity recognition.

Step 1. Download the dataset from https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones

Each datapoints consists of smartphone sensor measurements and your task is to predict the activity the person is performing

There are 561 features per data point and a total of 6 activities

The train and text split are already available.. ~/train/X_train, ~/train/y_train 

In [53]:
with open('human_activity_recognition/train/X_train.txt') as fs:
    for line in fs:
        features = line.strip().split()
        print(len(features))
        break

561


Step 2. Write Torch dataset class for the dataset 

In [ ]:
class HAR_dataset(Dataset):
    pass

Step 3. Create a neural network architecture for the task. (play around a bit with number of layers/number of nodes perlayer)

In [ ]:
class Classifier(nn.Module):
    pass

Step 4. Write a train function to train the model on the training set. 

In [ ]:
# save the model at the end of each epoch
def train(clf, train_data, batch_size, epochs, learning_rate):
    '''
    clf: classifier model
    train_data: pytorch dataset
    '''
    # define the optimizer 
    # define the loss criteria
    for _ in tqdm(range(epochs)): # the models are trained over multiple epochs..
        train_dataloader = DataLoader(train_data, batch_size=batch_size)
        for d, l in train_dataloader:
            ...
            ...

Step 5. Write a test function which loads a model and computes the accuracy on the test set

In [ ]:
def evaluate(model_path, test_data):
    '''
    model_path: Path to the saved model
    test_data: pytorch dataset
    '''
    # clf: load model from model_path
    test_dataloader = DataLoader(test_data, batch_size=100) # evaluate over a batch of examples which reduces time
    clf.eval()
    for d, l in test_dataloader:
        ...
        ...

In [ ]:
#Putting everything together
clf = Classifier()
train_data = HAR_dataset()
test_data = HAR_dataset()
batch_size = 
epochs = 
train(clf, train_data, batch_size, epochs)
evaluate(model_path, test_data)